In [1]:
# Import packages
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import h5py
import requests
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es

module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    
import modules.reflectance as refl

/Users/yheidiyoon/earth-analytics/capstone-project-test


In [2]:
# Get NEON reflectance data from NEON API
# Endpoints are data product, site, date, and release year
neon_base = "https://data.neonscience.org/api/v0/data"
data_product = "/DP3.30006.001"
site = "/GRSM"
post_fire_release = "/2017-10?package=basic&release=RELEASE-2022"
post_fire_url = neon_base + data_product + site + post_fire_release
post_fire_data = requests.get(post_fire_url)

# Similarly for the pre-fire data
pre_fire_release = "/2016-06?package=basic&release=RELEASE-2022"
pre_fire_url = neon_base + data_product + site + pre_fire_release
pre_fire_data = requests.get(pre_fire_url)

In [3]:
# Get NEON reflectance data
post_fire_url = (
    'https://storage.googleapis.com/neon-aop-products/2017/FullSite/D07/2017_GRSM_3/L3/Spectrometer/Reflectance/NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')
pre_fire_url = (
    'https://storage.googleapis.com/neon-aop-products/2016/FullSite/D07/2016_GRSM_2/L3/Spectrometer/Reflectance/NEON_D07_GRSM_DP3_274000_3947000_reflectance.h5')

post_data = et.data.get_data(url=post_fire_url)
pre_data = et.data.get_data(url=pre_fire_url)

In [4]:
#print(module_path)

In [5]:
# # Set working directory
# os.chdir(os.path.join(
#     et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads'))

# Data paths
#post_fire_path = os.path.join(et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads', 'NEON_D07_GRSM_DP3_274000_3947000_201710_reflectance.h5')
#pre_fire_path = os.path.join(et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads', 'NEON_D07_GRSM_DP3_274000_3947000_201606_reflectance.h5')

In [6]:
post_fire_refl, post_fire_metadata = refl.aop_h5refl2array(post_data)
pre_fire_refl, pre_fire_metadata = refl.aop_h5refl2array(pre_data)

In [7]:
pre_fire_stack = refl.stack_subset_bands(
    pre_fire_refl, pre_fire_metadata, (58, 84, 117, 400))

In [8]:
post_fire_stack = refl.stack_subset_bands(
    post_fire_refl, post_fire_metadata, (58, 84, 117, 400))

In [9]:
# Calculate NBR
post_fire_nbr = (
    post_fire_stack[:, :, 2] - post_fire_stack[:, :, 3]) / (
    post_fire_stack[:, :, 2] + post_fire_stack[:, :, 3])

pre_fire_nbr = (
    pre_fire_stack[:, :, 2] - pre_fire_stack[:, :, 3]) / (
    pre_fire_stack[:, :, 2] + pre_fire_stack[:, :, 3])

# Calculate dNBR
dnbr = pre_fire_nbr - post_fire_nbr

In [ ]:
# Plot dNBR
fig, ax = plt.subplots(figsize=(12, 6))

plot = plt.imshow(dnbr,
                  extent=post_fire_metadata['extent'],
                  cmap='PiYG')

cbar = plt.colorbar(plot, aspect=40)
cbar.set_label('Reflectance', rotation=90, labelpad=20)

# Remove scientific notation from tile coordinates
ax.ticklabel_format(useOffset=False, style='plain')

ax.set(title='Chimney Tops 2 Fire dNBR\n (June 2016-October 2017)',
       xlabel='UTM easting (m)',
       ylabel='UTM northing (m)')
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90)